In [ ]:
using Revise
using Pkg
Pkg.activate("..")
Pkg.instantiate()
Pkg.resolve()

include("../src/ChemQHam.jl")
using .ChemQHam

using ITensorChemistry
using SparseArrays
using LinearAlgebra


  Activating project at `~/Desktop/Felman/projects/Quantum/ChemQHam`
Precompiling project...
   9592.1 ms  ✓ ChemQHam
  1 dependency successfully precompiled in 12 seconds. 229 already precompiled.
  No Changes to `~/Desktop/Felman/projects/Quantum/ChemQHam/Project.toml`
  No Changes to `~/Desktop/Felman/projects/Quantum/ChemQHam/Manifest.toml`


### Test Hopping term a1c2

In [2]:
#### a1c2 ####

n_sites=2
init_ord = collect(1:n_sites)
algo="Hungarian"
ops_tol= 1e-8
ops_factor = 1e0

a_loc = 1
c_loc = 2

println("Testing a1c2...")

# ITensorChemistry opsum generation

molecule = Molecule([("Li", 0.00, 0.00, 0.0000), ("H", 0.00, 0.00, 1.000)])
(; opsum, h1e, h2e, nuc_e) = ITChem_opsum(molecule; n_sites=n_sites, idx_factor_list=nothing, it_terms_range=nothing, ops_tol=ops_tol, ops_factor=ops_factor)
aacc_sites = map(op->[local_op.sites[1] for local_op in op.args[2]], opsum)
it_terms_range = findall(sites->length(sites)==2 && sites[1] == a_loc && sites[2] == c_loc, aacc_sites)
opsum = reduce(+, op for op in opsum[it_terms_range]) * ops_factor

println("opsum: ", opsum)


# TensorKit opsum generation

terms = gen_ChemOpSum(h1e, h2e, nuc_e, init_ord; tol=ops_tol)

terms_range = findall(t->length(t.sites)==2 && t.sites[1] == a_loc && t.sites[2] == c_loc , terms)

op_terms=terms[terms_range] * ops_factor
println("op_terms: ", op_terms)


Testing a1c2...
opsum: sum(
  0.14792631902885367 c†↑(1,) c↑(2,)
  0.14792631902885367 c†↓(1,) c↓(2,)
)
op_terms: ChemOpSum with 1 terms:
  1. [ a1 ]_1 ── [ c1 ]_2    factor = 0.14792631902885367


In [10]:
# TensorKit opsum to TensorKit MPO and sparse mat
table, primary_ops, factors = terms_to_table(op_terms)
symbolic_mpo, virt_spaces = construct_symbolic_mpo(table, primary_ops, factors; verbose=false)
mpo = symbolic_to_tensorkit_mpo(symbolic_mpo, virt_spaces, "u1su2"; verbose=false)
tk_mat = mpo_to_mat(mpo)
tk_nz = collect(zip(findnz(tk_mat)...))

# ITensorChemistry opsum to sparse mat
it_mat = ITChem_mat(opsum, n_sites)
it_nz = collect(zip(findnz(it_mat)...))

# compare
cond_a1c2 = isapprox(norm(tk_mat-it_mat), 0.0, atol=1e-6)
println("Condition a1c2: ", cond_a1c2)


Condition a1c2: true
